In [1]:

import sys
sys.path.append('src/')
from src.logic_utils import get_lang

import torch
import numpy as np

In [2]:
lark_path = '../src/lark/exp.lark'
lang_base_path = '../data/lang/'
lang, clauses, bk, atoms = get_lang(
        lark_path, lang_base_path,'coinjump', 'coinjump')

In [3]:
atoms

[.(__F__),
 .(__T__),
 closeby(obj1,obj2),
 closeby(obj1,obj3),
 closeby(obj1,obj4),
 closeby(obj2,obj1),
 closeby(obj2,obj3),
 closeby(obj2,obj4),
 closeby(obj3,obj1),
 closeby(obj3,obj2),
 closeby(obj3,obj4),
 closeby(obj4,obj1),
 closeby(obj4,obj2),
 closeby(obj4,obj3),
 have_key(img),
 jump(img),
 left_go_get_key(img),
 left_go_to_door(img),
 not_have_key(img),
 on_left(obj1,obj2),
 on_left(obj1,obj3),
 on_left(obj1,obj4),
 on_left(obj2,obj1),
 on_left(obj2,obj3),
 on_left(obj2,obj4),
 on_left(obj3,obj1),
 on_left(obj3,obj2),
 on_left(obj3,obj4),
 on_left(obj4,obj1),
 on_left(obj4,obj2),
 on_left(obj4,obj3),
 on_right(obj1,obj2),
 on_right(obj1,obj3),
 on_right(obj1,obj4),
 on_right(obj2,obj1),
 on_right(obj2,obj3),
 on_right(obj2,obj4),
 on_right(obj3,obj1),
 on_right(obj3,obj2),
 on_right(obj3,obj4),
 on_right(obj4,obj1),
 on_right(obj4,obj2),
 on_right(obj4,obj3),
 right_go_get_key(img),
 right_go_to_door(img),
 type(obj1,agent),
 type(obj1,door),
 type(obj1,enemy),
 type(obj1,k

In [4]:

from percept import SlotAttentionPerceptionModule, YOLOPerceptionModule
from facts_converter import FactsConverter
from nsfr import NSFReasoner
from logic_utils import build_infer_module, generate_atoms
from valuation import RLValuationModule
device = torch.device('cpu')

In [5]:
def get_nsfr_model(lang, clauses, atoms, bk, device):
    PM = YOLOPerceptionModule(e=4, d=11, device=device)
    VM = RLValuationModule(
            lang=lang, device=device)
    FC = FactsConverter(lang=lang, perception_module=PM,
                        valuation_module=VM, device=device)
    IM = build_infer_module(clauses, atoms, lang,
                            m=len(clauses), infer_step=16, device=device)
    # Neuro-Symbolic Forward Reasoner
    NSFR = NSFReasoner(perception_module=PM, facts_converter=FC,
                       infer_module=IM, atoms=atoms, bk=bk, clauses=clauses)
    return NSFR

In [6]:
NSFR = get_nsfr_model(lang, clauses, atoms, bk, device=device)

In [7]:
metric =[[1,0,0,0,11,2],[0,1,0,0,14,2],[0,0,1,0,14.1,2],[0,0,0,1,20,2]]
x = torch.tensor(np.array(metric), dtype=torch.float32).unsqueeze(0)

In [8]:
x.shape

torch.Size([1, 4, 6])

In [9]:
V_T = NSFR(x)

In [10]:
NSFR.print_valuation_batch(V_T)

====== LEARNED PROGRAM ======
C_0:  jump(X):-type(O1,agent),type(O2,enemy),closeby(O1,O2). 1.0
C_1:  left_go_get_key(X):-type(O1,agent),type(O2,key),on_left(O2,O1),not_have_key(X). 1.0
C_2:  right_go_get_key(X):-type(O1,agent),type(O2,key),on_right(O2,O1),not_have_key(X). 1.0
C_3:  left_go_to_door(X):-type(O1,agent),type(O2,door),on_left(O2,O1),have_key(X). 1.0
C_4:  right_go_to_door(X):-type(O1,agent),type(O2,door),on_right(O2,O1),have_key(X). 1.0
===== BATCH:  0 =====
45 type(obj1,agent) :  1.0
1 .(__T__) :  1.0
54 type(obj3,door) :  1.0
59 type(obj4,enemy) :  1.0
52 type(obj2,key) :  1.0
6 closeby(obj2,obj3) :  0.996
9 closeby(obj3,obj2) :  0.996
25 on_left(obj3,obj1) :  0.9
26 on_left(obj3,obj2) :  0.9
28 on_left(obj4,obj1) :  0.9
29 on_left(obj4,obj2) :  0.9
30 on_left(obj4,obj3) :  0.9
32 on_right(obj1,obj3) :  0.9
33 on_right(obj1,obj4) :  0.9
35 on_right(obj2,obj3) :  0.9
36 on_right(obj2,obj4) :  0.9
22 on_left(obj2,obj1) :  0.9
31 on_right(obj1,obj2) :  0.9
39 on_right(obj3,o

In [11]:
V_T

tensor([[0.06869, 1.00000, 0.06869, 0.06869, 0.06869, 0.06869, 0.99623, 0.06869, 0.06869, 0.99623, 0.06869, 0.06869, 0.06869, 0.06869, 0.06869, 0.09013, 0.83773, 0.08477, 0.90000, 0.06869, 0.06869, 0.06869, 0.90000, 0.06869, 0.06869, 0.90000, 0.90000, 0.06869, 0.90000, 0.90000, 0.90000, 0.90000, 0.90000, 0.90000,
         0.06869, 0.90000, 0.90000, 0.06869, 0.06869, 0.90000, 0.06869, 0.06869, 0.06869, 0.09120, 0.06884, 1.00000, 0.06869, 0.06869, 0.06869, 0.06869, 0.06869, 0.06869, 1.00000, 0.06869, 1.00000, 0.06869, 0.06869, 0.06869, 0.06869, 1.00000, 0.06869]])

In [12]:
atoms

[.(__F__),
 .(__T__),
 closeby(obj1,obj2),
 closeby(obj1,obj3),
 closeby(obj1,obj4),
 closeby(obj2,obj1),
 closeby(obj2,obj3),
 closeby(obj2,obj4),
 closeby(obj3,obj1),
 closeby(obj3,obj2),
 closeby(obj3,obj4),
 closeby(obj4,obj1),
 closeby(obj4,obj2),
 closeby(obj4,obj3),
 have_key(img),
 jump(img),
 left_go_get_key(img),
 left_go_to_door(img),
 not_have_key(img),
 on_left(obj1,obj2),
 on_left(obj1,obj3),
 on_left(obj1,obj4),
 on_left(obj2,obj1),
 on_left(obj2,obj3),
 on_left(obj2,obj4),
 on_left(obj3,obj1),
 on_left(obj3,obj2),
 on_left(obj3,obj4),
 on_left(obj4,obj1),
 on_left(obj4,obj2),
 on_left(obj4,obj3),
 on_right(obj1,obj2),
 on_right(obj1,obj3),
 on_right(obj1,obj4),
 on_right(obj2,obj1),
 on_right(obj2,obj3),
 on_right(obj2,obj4),
 on_right(obj3,obj1),
 on_right(obj3,obj2),
 on_right(obj3,obj4),
 on_right(obj4,obj1),
 on_right(obj4,obj2),
 on_right(obj4,obj3),
 right_go_get_key(img),
 right_go_to_door(img),
 type(obj1,agent),
 type(obj1,door),
 type(obj1,enemy),
 type(obj1,k

In [13]:
for i, atom in enumerate(atoms):
    print(V_T[0][i], atom)

tensor(0.06869) .(__F__)
tensor(1.) .(__T__)
tensor(0.06869) closeby(obj1,obj2)
tensor(0.06869) closeby(obj1,obj3)
tensor(0.06869) closeby(obj1,obj4)
tensor(0.06869) closeby(obj2,obj1)
tensor(0.99623) closeby(obj2,obj3)
tensor(0.06869) closeby(obj2,obj4)
tensor(0.06869) closeby(obj3,obj1)
tensor(0.99623) closeby(obj3,obj2)
tensor(0.06869) closeby(obj3,obj4)
tensor(0.06869) closeby(obj4,obj1)
tensor(0.06869) closeby(obj4,obj2)
tensor(0.06869) closeby(obj4,obj3)
tensor(0.06869) have_key(img)
tensor(0.09013) jump(img)
tensor(0.83773) left_go_get_key(img)
tensor(0.08477) left_go_to_door(img)
tensor(0.90000) not_have_key(img)
tensor(0.06869) on_left(obj1,obj2)
tensor(0.06869) on_left(obj1,obj3)
tensor(0.06869) on_left(obj1,obj4)
tensor(0.90000) on_left(obj2,obj1)
tensor(0.06869) on_left(obj2,obj3)
tensor(0.06869) on_left(obj2,obj4)
tensor(0.90000) on_left(obj3,obj1)
tensor(0.90000) on_left(obj3,obj2)
tensor(0.06869) on_left(obj3,obj4)
tensor(0.90000) on_left(obj4,obj1)
tensor(0.90000) on_le